In [1]:
from processing import *
import models
import torch

train_prop = 0.9
num_files = 100_000
data_dir = "../Data3"
max_batch_size = 128
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

# Define model
model1 = models.TransformerModel1()
model1.load_state_dict(torch.load("site1_model.pth", map_location=torch.device(device)))
model1 = model1.to(device)
model1.eval()
# Define model
model2 = models.TransformerModel1()
model2.load_state_dict(torch.load("site2_model.pth", map_location=torch.device(device)))
model2 = model2.to(device)
model2.eval()
# Define model
model12 = models.TransformerModel1()
model12.load_state_dict(torch.load("site12_model.pth", map_location=torch.device(device)))
model12 = model12.to(device)
model12.eval()

# Recreate test indices
torch.manual_seed(random_seed)
ind = int(train_prop * num_files)
idx = torch.randperm(num_files)
test_indices = idx[ind:].tolist()

# Load data from test indices
X = [convert_files(data_dir + "/sampled_genotypes/sample_" + str(i), data_dir + "/commands/command_" + str(i)) for i in test_indices] 

C = [convert_command_file1(data_dir + "/commands/command_" + str(i)) for i in test_indices]
C = [c for x,c in zip(X,C) if x is not None]

X = [x for x in X if x is not None]

X = torch.tensor(X) - 1
X = X.reshape(-1, sample_width*num_chrom * 2)

C = torch.tensor(C)
C = C.reshape(-1, 11)

y = torch.tensor([i % 2 for i in range(1, X.shape[0] + 1)])

print(X.shape)

cuda
40000
_5000
45000
80000
68000
67000
47000
49000
76000
75000
72000
torch.Size([20000, 11000])


In [4]:
with torch.no_grad():
    y_pred = torch.zeros(X.shape[0])
    for istart in range(0, X.shape[0], max_batch_size):
        iend = min(istart+max_batch_size,X.shape[0])
        X_batch = X[istart:iend].to(device)
        y_pred[istart:iend] = model12(X_batch)        

    weak_epistasis = (C[:,7] < C[:,10])
    weak_count = weak_epistasis.sum().item()

    # loss = criterion(y_pred, y.float())
    predictions = (y_pred >= 0.5).int()

    print()
    print("full set:")

    num_correct = (predictions == y).sum().item()
    print(f"Accuracy: {num_correct/y.shape[0]:0.4f}")

    print("\t0\t1")
    for i in range(2):
        called_0 = torch.sum((predictions == 0) & (y == i)).item()
        called_1 = torch.sum((predictions == 1) & (y == i)).item()
        sums = called_0 + called_1
        called_0 /= sums
        called_1 /= sums
        print(i, f"{called_0:.3f}" , f"{called_1:.3f}", sep='\t')
        
    called_epistasic = (predictions == 1)

    print()
    print("M and T evaluation")

    for i in range(8):
        minim = 0.1 + i/10
        maxum = 0.2 + i/10

        subset_0 = ((C[:,1] >= minim) & (C[:,1] < maxum))

        for j in range(10):

            minim = 100 + j*50
            maxum = 150 + j*50

            subset_1 = ((C[:,2] >= minim) & (C[:,2] < maxum))

            subset = (subset_0 & subset_1) # & called_epistasic)

            #Counting proportion of those called epistasis which are correct

            subset_count = subset.sum().item()
            num_correct = ((predictions == y) & subset).sum().item()

            if subset_count == 0:
                print("0",end='')
                continue
            
            ratio = num_correct / subset_count

            if ratio < 0.5:
                print(".",end='')
            elif ratio < 0.66:
                print("-",end='')
            elif ratio < 0.80:
                print("+",end='')
            elif ratio < 0.95:
                print("$",end='')
            else:
                print("#",end='')
        
        print()


    print()

    print("epistrength and normstrength evaluation")

    for i in range(9):
        minim = 0.01 + i * 0.01
        maxum = 0.02 + i * 0.01

        subset_0 = ((C[:,7] >= minim) & (C[:,7] < maxum))

        for j in range(20):

            minim = -0.05  + j * 0.005
            maxum = -0.045 + j * 0.005

            subset_1 = ((C[:,10] >= minim) & (C[:,10] < maxum))

            subset = (subset_0 & subset_1) # & called_epistatic)

            subset_count = subset.sum().item()
            num_correct = ((predictions == y) & subset).sum().item()

            if subset_count == 0:
                print("0",end='')
                continue
            
            ratio = num_correct / subset_count

            if ratio < 0.5:
                print(".",end='')
            elif ratio < 0.66:
                print("-",end='')
            elif ratio < 0.80:
                print("+",end='')
            elif ratio < 0.95:
                print("$",end='')
            else:
                print("#",end='')
        
        print()


    print()


full set:
Accuracy: 0.8616
	0	1
0	0.799	0.201
1	0.076	0.924

M and T evaluation
$$$$$$$$$$
$$$$$$$$$$
$$$$$$$$$$
++$$$$$$$$
++$$$$$$$$
$$$$$$$$$$
$$$$$$$$$$
$$$$$$$$$$

epistrength and normstrength evaluation
++++$++++00+$$++++++
++$$$$$$$00$$$$$+$+$
$$$$$$$$$00$$$$$+$$$
$$$$$$$$$00$$$$$$$$$
$$$$$$$$$00$$#$$$$$$
$$$$$$$$$00$$$$$$$$$
$$$$$$$$$00$$$$$$$$$
$$$$$$$$$00$$$$$$$$$
$$$$$$$$$00$$#$$$$$$



In [9]:
with torch.no_grad():
    y_pred = torch.zeros(X.shape[0])
    for istart in range(0, X.shape[0], max_batch_size):
        iend = min(istart+max_batch_size,X.shape[0])
        X_batch = X[istart:iend].to(device)
        y_pred[istart:iend] = model12(X_batch)        

    num_correct = (y_pred[::2] - y_pred[1::2] > 0).sum().item() 

    print(f"Accuracy: {num_correct*2/y_pred.shape[0]:0.4f}")


Accuracy: 0.9464


In [10]:
with torch.no_grad():
    y_pred = torch.zeros(X.shape[0])
    for istart in range(0, X.shape[0], max_batch_size):
        iend = min(istart+max_batch_size,X.shape[0])
        X_batch = X[istart:iend].to(device)
        y_pred[istart:iend] = model1(X_batch)        

    num_correct = (y_pred[::2] - y_pred[1::2] > 0).sum().item() 

    print(f"Accuracy: {num_correct*2/y_pred.shape[0]:0.4f}")


Accuracy: 0.8687


In [11]:
with torch.no_grad():
    y_pred = torch.zeros(X.shape[0])
    for istart in range(0, X.shape[0], max_batch_size):
        iend = min(istart+max_batch_size,X.shape[0])
        X_batch = X[istart:iend].to(device)
        y_pred[istart:iend] = model2(X_batch)        

    num_correct = (y_pred[::2] - y_pred[1::2] > 0).sum().item() 

    print(f"Accuracy: {num_correct*2/y_pred.shape[0]:0.4f}")

Accuracy: 0.8684


In [12]:
with torch.no_grad():
    y_pred = torch.zeros(X.shape[0])
    for istart in range(0, X.shape[0], max_batch_size):
        iend = min(istart+max_batch_size,X.shape[0])
        X_batch = X[istart:iend].to(device)
        y_pred[istart:iend] = (model1(X_batch) + model2(X_batch))/2     

    num_correct = (y_pred[::2] - y_pred[1::2] > 0).sum().item()

    print(f"Accuracy: {num_correct*2/y_pred.shape[0]:0.4f}")

Accuracy: 0.9482


In [14]:
with torch.no_grad():
    y_pred1 = torch.zeros(X.shape[0])
    y_pred2 = torch.zeros(X.shape[0])
    for istart in range(0, X.shape[0], max_batch_size):
        iend = min(istart+max_batch_size,X.shape[0])
        X_batch = X[istart:iend].to(device)
        y_pred1[istart:iend] = model1(X_batch)
        y_pred2[istart:iend] = model2(X_batch)

    num_correct = ((y_pred1[::2] - y_pred1[1::2] > 0) | (y_pred2[::2] - y_pred2[1::2] > 0)).sum().item()

    print(f"Accuracy: {num_correct*2/y_pred.shape[0]:0.4f}")

Accuracy: 0.9754


In [16]:
with torch.no_grad():
    y_pred = torch.zeros(X.shape[0])
    for istart in range(0, X.shape[0], max_batch_size):
        iend = min(istart+max_batch_size,X.shape[0])
        X_batch = X[istart:iend].to(device)
        y_pred[istart:iend] = (model1(X_batch) + model2(X_batch) + model12(X_batch))/3

    num_correct = (y_pred[::2] - y_pred[1::2] > 0).sum().item()

    print(f"Accuracy: {num_correct*2/y_pred.shape[0]:0.4f}")

Accuracy: 0.9483


In [18]:
weak_epistasis = (C[:,7] < C[:,10])
weak_count = weak_epistasis.sum().item()

# loss = criterion(y_pred, y.float())
predictions = (y_pred >= 0.5).int()

print()
print("full set:")

num_correct = (predictions == y).sum().item()
print(f"Accuracy: {num_correct/y.shape[0]:0.4f}")

print("\t0\t1")
for i in range(2):
    called_0 = torch.sum((predictions == 0) & (y == i)).item()
    called_1 = torch.sum((predictions == 1) & (y == i)).item()
    sums = called_0 + called_1
    called_0 /= sums
    called_1 /= sums
    print(i, f"{called_0:.3f}" , f"{called_1:.3f}", sep='\t')
    
called_epistasic = (predictions == 1)

print()
print("M and T evaluation")

for i in range(8):
    minim = 0.1 + i/10
    maxum = 0.2 + i/10

    subset_0 = ((C[:,1] >= minim) & (C[:,1] < maxum))

    for j in range(10):

        minim = 100 + j*50
        maxum = 150 + j*50

        subset_1 = ((C[:,2] >= minim) & (C[:,2] < maxum))

        subset = (subset_0 & subset_1 & called_epistasic)

        #Counting proportion of those called epistasis which are correct

        subset_count = subset.sum().item()
        num_correct = ((predictions == y) & subset).sum().item()

        if subset_count == 0:
            print("0",end='')
            continue
        
        ratio = num_correct / subset_count

        if ratio < 0.5:
            print(".",end='')
        elif ratio < 0.66:
            print("-",end='')
        elif ratio < 0.80:
            print("+",end='')
        elif ratio < 0.95:
            print("$",end='')
        else:
            print("#",end='')
    
    print()


print()

print("epistrength and normstrength evaluation")

for i in range(9):
    minim = 0.01 + i * 0.01
    maxum = 0.02 + i * 0.01

    subset_0 = ((C[:,7] >= minim) & (C[:,7] < maxum))

    for j in range(20):

        minim = -0.05  + j * 0.005
        maxum = -0.045 + j * 0.005

        subset_1 = ((C[:,10] >= minim) & (C[:,10] < maxum))

        subset = (subset_0 & subset_1 & called_epistasic)

        subset_count = subset.sum().item()
        num_correct = ((predictions == y) & subset).sum().item()

        if subset_count == 0:
            print("0",end='')
            continue
        
        ratio = num_correct / subset_count

        if ratio < 0.5:
            print(".",end='')
        elif ratio < 0.66:
            print("-",end='')
        elif ratio < 0.80:
            print("+",end='')
        elif ratio < 0.95:
            print("$",end='')
        else:
            print("#",end='')
    
    print()


print()


full set:
Accuracy: 0.8618
	0	1
0	0.758	0.242
1	0.034	0.966

M and T evaluation
$$$$$$$$$$
$$+$$$$$$+
++++$$$$$$
+++++++$++
+++$++++++
+$+$++$$++
++$$+$$$$$
$$+$$+$$$$

epistrength and normstrength evaluation
++++$$+$$00$$$++++++
++$$$$$$$00$$$$++$++
+$++$$+$$00$$$++++$$
$+$$+$$$+00+$+++++++
+$+$$+$$$00+$$$$++++
$++$$$$$+00$$+$+++$+
+$+$$$$$+00+$$++$+++
+$$+$+$$$00+$$+$$$++
+$+$+$$$+00$+$$$+$++



In [32]:
def ff2(sampling_file, command_file):
    with open(sampling_file, "r") as f:
        lines = f.readlines()

    X = [[int(l) for l in line[:-1]] for line in lines]

    with open(command_file, "r") as f:
        s = f.readlines()[0].split()

    points = [float(s[6]), float(s[7]), float(s[10])]
    ind1, ind2, regular_ind = [round(num_samples*point - 0.5) for point in points]
    for ind in [ind1,ind2,regular_ind]:
        print(sum([x[ind] for x in X]) - 100)

def foo():
    for _ in range(10):
        idx = int(random.random() * 100_000)
        c = convert_command_file1(data_dir + "/commands/command_" + str(idx))
        print(c[0][10])
        ff2(data_dir + "/sampled_genotypes/sample_" + str(idx), data_dir + "/commands/command_" + str(idx))
foo()
    

-0.025209760672355145
23
100
98
0.02447867113227701
26
97
-43
-0.027080472086715317
-100
-44
89
-0.015909187820655146
-12
99
90
0.02501946455532246
23
100
-91
0.0206096803181641
55
100
-94
-0.043667937250996935
-42
98
94
0.008337755332822577
-89
-48
-70
0.04540667612492686
-97
57
-98
-0.011578182677814833
75
97
91
